<a href="https://colab.research.google.com/github/WellcomePeujio/Calculo-del-Retorno-de-una-Estrategia-de-Trading-con-Backtesting/blob/main/C%C3%A1lculo_del_Retorno_de_una_Estrategia_de_Trading_con_Backtesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cálculo del Retorno de una Estrategia de Trading con Backtesting

**Título**: "¿Funciona realmente tu estrategia de trading?"

**Problema**: Se prueba una estrategia basada en **medias móviles** sobre los últimos **3 años** en acciones de **Tesla (TSLA)**.  
El objetivo es evaluar si la estrategia genera un retorno superior al mercado y comparar:

- **Retorno real** de la estrategia aplicada.
- **Retorno esperado** basado en el rendimiento histórico del activo.

**Visualización**:  
Genera un gráfico mostrando los **retornos históricos** de TSLA y la **estrategia aplicada** para visualizar su efectividad.


# Resolución Matemática: Cálculo del Retorno de una Estrategia de Trading con Backtesting

## Paso 1: Definición del Retorno de la Estrategia

El rendimiento de una estrategia de trading se calcula con la siguiente fórmula:

$$
R = \frac{P_f - P_i}{P_i} \times 100
$$

Donde:
- $R$ es el **retorno** de la estrategia en porcentaje.
- $P_f$ es el **precio final** del activo después del período de inversión.
- $P_i$ es el **precio inicial** del activo.

Si la estrategia incluye múltiples operaciones, el retorno acumulado se calcula con:

$$
R_{\text{total}} = \prod_{t=1}^{N} (1 + R_t) - 1
$$

Donde:
- $R_t$ es el **retorno diario** en el tiempo $t$.
- $N$ es el número total de días en el análisis.

---

## Paso 2: Estrategia Basada en Medias Móviles

Para implementar la estrategia de medias móviles, definimos:

1. **Media móvil rápida ($SMA_{corto}$):** Promedio de precios en un período corto (ej. 20 días).
2. **Media móvil lenta ($SMA_{largo}$):** Promedio de precios en un período más largo (ej. 50 días).

Las señales de trading se generan así:

- **Compra ($\text{BUY}$) cuando**:  
  $$ SMA_{corto}(t) > SMA_{largo}(t) $$
- **Venta ($\text{SELL}$) cuando**:  
  $$ SMA_{corto}(t) < SMA_{largo}(t) $$

---

## Paso 3: Comparación con el Retorno del Mercado

Para evaluar si la estrategia supera al mercado, comparamos su retorno con el **buy & hold**, que se calcula como:

$$
R_{\text{mercado}} = \frac{P_f - P_i}{P_i} \times 100
$$

Si:

$$
R_{\text{estrategia}} > R_{\text{mercado}}
$$

La estrategia es efectiva y genera un mejor rendimiento.

---

## Paso 4: Análisis de Riesgo

Evaluamos la **máxima caída (drawdown)** de la estrategia:

$$
\text{Drawdown}(t) = \frac{P_{\max} - P_t}{P_{\max}} \times 100
$$

Donde $P_{\max}$ es el precio máximo alcanzado antes de una caída. Un drawdown alto indica una estrategia más riesgosa.

---

## Paso 5: Conclusión

1. Calculamos el **retorno real** de la estrategia.
2. Comparamos con el **retorno del mercado** (buy & hold).
3. Evaluamos la **volatilidad y drawdown** para medir el riesgo.
4. Visualizamos la estrategia con un gráfico de precios, medias móviles y señales de compra/venta.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

# Descargar datos de Tesla (TSLA) de los últimos 3 años
symbol = "TSLA"
data = yf.download(symbol, period="3y")

# Cálculo de medias móviles
data["SMA_20"] = data["Close"].rolling(window=20).mean()
data["SMA_50"] = data["Close"].rolling(window=50).mean()

# Generar señales de compra y venta
data["Signal"] = 0  # 0: No posición
data.loc[data["SMA_20"] > data["SMA_50"], "Signal"] = 1  # Compra
data.loc[data["SMA_20"] < data["SMA_50"], "Signal"] = -1  # Venta

# Simulación del rendimiento de la estrategia
data["Returns"] = data["Close"].pct_change()
data["Strategy"] = data["Signal"].shift(1) * data["Returns"]

# Calcular retorno acumulado
data["Cumulative_Market"] = (1 + data["Returns"]).cumprod()
data["Cumulative_Strategy"] = (1 + data["Strategy"]).cumprod()

# Gráfica de evolución del precio con medias móviles y señales
plt.figure(figsize=(12, 6))
plt.plot(data["Close"], label="Precio TSLA", color="black", alpha=0.7)
plt.plot(data["SMA_20"], label="SMA 20", color="blue", linestyle="--")
plt.plot(data["SMA_50"], label="SMA 50", color="red", linestyle="--")

# Señales de compra y venta
buy_signals = data[data["Signal"] == 1]
sell_signals = data[data["Signal"] == -1]
plt.scatter(buy_signals.index, buy_signals["Close"], marker="^", color="green", label="Compra", alpha=1, zorder=3)
plt.scatter(sell_signals.index, sell_signals["Close"], marker="v", color="red", label="Venta", alpha=1, zorder=3)

plt.title("Estrategia de Medias Móviles en Tesla (TSLA)")
plt.xlabel("Fecha")
plt.ylabel("Precio de Cierre ($)")
plt.legend()
plt.grid(True)
plt.show()

# Gráfica del rendimiento acumulado
plt.figure(figsize=(12, 6))
plt.plot(data["Cumulative_Market"], label="Buy & Hold", color="black", linestyle="--")
plt.plot(data["Cumulative_Strategy"], label="Estrategia SMA", color="blue")
plt.title("Comparación de Rendimiento: Estrategia vs. Mercado")
plt.xlabel("Fecha")
plt.ylabel("Retorno Acumulado")
plt.legend()
plt.grid(True)
plt.show()